In [2]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
%%time
df = pd.read_feather("../data/merged_click_data", nthreads=)

CPU times: user 13.1 s, sys: 9.23 s, total: 22.3 s
Wall time: 5.96 s


In [3]:
%%time
click_data = pd.read_hdf("../data/click_data.hdf", "click_data")

CPU times: user 10.8 s, sys: 11.1 s, total: 21.9 s
Wall time: 21.8 s


In [4]:
%%time
click_data["rank_by_ip"] = click_data.groupby("ip").cumcount()

CPU times: user 31.2 s, sys: 8.58 s, total: 39.8 s
Wall time: 39.8 s


In [5]:
rank_by_ip=click_data[["rank_by_ip"]]

In [6]:
rank_by_ip.to_hdf("../data/rank_by_ip.hdf", "rank_by_ip")

In [7]:
%%time
ip_os_device = pd.read_hdf("../data/ip_os_device.hdf", "ip_os_device")

CPU times: user 13.6 s, sys: 7.45 s, total: 21 s
Wall time: 21 s


In [8]:
%%time
ip_os_device_dow = pd.read_hdf("../data/ip_os_device_dow.hdf", "ip_os_device_dow")

CPU times: user 4 ms, sys: 2.09 s, total: 2.1 s
Wall time: 2.09 s


In [9]:
%%time
click_data = pd.concat([click_data, ip_os_device, ip_os_device_dow], axis=1)

CPU times: user 10.4 s, sys: 5.44 s, total: 15.8 s
Wall time: 15.8 s


In [10]:
%%time
click_data["rank_by_ip_os_device"] = click_data.groupby("ip_os_device").cumcount()
click_data["rank_by_ip_os_device_dow"] = click_data.groupby("ip_os_device_dow").cumcount()

CPU times: user 2min 10s, sys: 27.1 s, total: 2min 37s
Wall time: 2min 37s


In [11]:
%%time
click_data[["rank_by_ip", "rank_by_ip_os_device", "rank_by_ip_os_device_dow"]].to_hdf("../data/ranks.hdf", "ranks")

CPU times: user 5.51 s, sys: 10.5 s, total: 16 s
Wall time: 16 s


In [12]:
time_ip = click_data[["click_time", "ip"]]

In [13]:
%%time
time_ip.sort_values(by=["ip", "click_time"], inplace=True)

CPU times: user 3min 4s, sys: 14.3 s, total: 3min 18s
Wall time: 3min 18s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
%%time
time_ip["prev_time"] = time_ip["click_time"].shift()
time_ip["prev_ip"] = time_ip["ip"].shift()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


CPU times: user 1.54 s, sys: 1.99 s, total: 3.54 s
Wall time: 3.53 s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
%%time
time_ip["click_time_interval_by_ip"] = time_ip["click_time"] - time_ip["prev_time"]

CPU times: user 58.8 s, sys: 3.56 s, total: 1min 2s
Wall time: 1min 1s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
time_ip["click_time_interval_by_ip"]=time_ip["click_time_interval_by_ip"].where(time_ip["ip"] == time_ip["prev_ip"], np.nan)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
%%time
time_ip.sort_index(inplace=True)

CPU times: user 44.6 s, sys: 7.21 s, total: 51.8 s
Wall time: 51.8 s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
%%time
time_ip[["click_time_interval_by_ip"]].to_hdf("../data/click_time_interval_by_ip.hdf", "click_time_interval_by_ip")

CPU times: user 980 ms, sys: 3.14 s, total: 4.12 s
Wall time: 4.12 s


In [19]:
del time_ip
gc.collect()

42

In [20]:
%%time
time_ip_os_device = click_data[["click_time", "ip_os_device"]]
time_ip_os_device.sort_values(by=["ip_os_device", "click_time"], inplace=True)
time_ip_os_device["prev_time"] = time_ip_os_device["click_time"].shift()
time_ip_os_device["prev_ip_os_device"] = time_ip_os_device["ip_os_device"].shift()
time_ip_os_device["click_time_interval_by_ip_os_device"] = time_ip_os_device["click_time"] - time_ip_os_device["prev_time"]
time_ip_os_device["click_time_interval_by_ip_os_device"] = time_ip_os_device["click_time_interval_by_ip_os_device"].where(time_ip_os_device["ip_os_device"] == time_ip_os_device["prev_ip_os_device"], np.nan)
time_ip_os_device.sort_index(inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

CPU times: user 9min 35s, sys: 30.7 s, total: 10min 6s
Wall time: 10min 3s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [21]:
%%time
time_ip_os_device[["click_time_interval_by_ip_os_device"]].to_hdf("../data/click_time_interval_by_ip_os_device.hdf", "click_time_interval_by_ip_os_device")

CPU times: user 1 s, sys: 3.11 s, total: 4.11 s
Wall time: 4.1 s


In [ ]:
%%time
time_dow = click_data[["click_time", "ip_os_device_dow"]]
time_dow.sort_values(by=["ip_os_device_dow", "click_time"], inplace=True)
time_dow["prev_time"] = time_dow["click_time"].shift()
time_dow["prev_ip_os_device_dow"] = time_dow["ip_os_device_dow"].shift()
time_dow["click_time_interval_by_ip_os_device_dow"] = time_dow["click_time"] - time_dow["prev_time"]
time_dow["click_time_interval_by_ip_os_device_dow"] = time_dow["click_time_interval_by_ip_os_device_dow"].where(time_dow["ip_os_device_dow"] == time_dow["prev_ip_os_device_dow"], np.nan)
time_dow.sort_index(inplace=True)
time_dow[["click_time_interval_by_ip_os_device_dow"]].to_hdf("../data/click_time_interval_by_ip_os_device_dow.hdf", "click_time_interval_by_ip_os_device_dow")

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

CPU times: user 5min 21s, sys: 2min 45s, total: 8min 7s
Wall time: 8min 5s


In [ ]:
%%time
time_hour = click_data[["click_time", "ip_os_device_dow_hour"]]
time_hour.sort_values(by=["ip_os_device_dow_hour", "click_time"], inplace=True)
time_hour["prev_time"] = time_hour["click_time"].shift()
time_hour["prev_ip_os_device_dow_hour"] = time_hour["ip_os_device_dow_hour"].shift()
time_hour["click_time_interval_by_ip_os_device_dow_hour"] = time_hour["click_time"] - time_hour["prev_time"]
time_hour["click_time_interval_by_ip_os_device_dow_hour"] = time_hour["click_time_interval_by_ip_os_device_dow_hour"].where(time_hour["ip_os_device_dow_hour"] == time_hour["prev_ip_os_device_dow_hour"], np.nan)
time_hour.sort_index(inplace=True)
time_hour[["click_time_interval_by_ip_os_device_dow_hour"]].to_hdf("../data/click_time_interval_by_ip_os_device_dow_hour.hdf", "click_time_interval_by_ip_os_device_dow_hour")

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p